## Automated Project: Planning, Estimation, and Allocation using Sambanova and CrewAI

# Initial Imports & Sambanova Setup

Sign up for a free API key at https://cloud.sambanova.ai/

In [1]:
SAMBANOVA_BASE_URL = "https://api.sambanova.ai/v1"
SAMBANOVA_API_KEY="INSERT_YOUR_API_KEY_HERE"

In [2]:
import os
import sys


In [3]:
from crewai import LLM
# # # Set Up Sambanova

from utils import get_sambanova_api_key, update_task_output_format
llm = LLM(model="sambanova/Meta-Llama-3.1-405B-Instruct", api_key=get_sambanova_api_key(), base_url=os.getenv("SAMBANOVA_BASE_URL"))


In [4]:
# Warning control
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv
load_dotenv()

import os
import yaml
from crewai import Agent, Task, Crew

## Loading Tasks and Agents YAML files

In [5]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Create Pydantic Models for Structured Output

In [6]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")




In [7]:
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate

parser = PydanticOutputParser(pydantic_object=ProjectPlan)  
format_instructions = parser.get_format_instructions()

#Helper Function for Structured Output
tasks_config = update_task_output_format(tasks_config, 'resource_allocation', format_instructions)



In [8]:
tasks_config

{'task_breakdown': {'description': "Carefully analyze the project_requirements for the {project_type} project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:\n{project_requirements}\n\nTeam members:\n{team_members}\n",
  'expected_output': 'A comprehensive list of tasks with detailed descriptions, timelines, dependencies, and deliverables. Your final output MUST include a Gantt chart or similar timeline visualization specific to the {project_type} project.\n'},
 'time_resource_estimation': {'description': 'Thoroughly evaluate each task in the {project_type} project to estimate the time, resources, and effort required. Use historical data, task complexity, and available resources to provide a realistic estimation for each task.\n',
  'expected_output': 'A detailed estimation report outlining the time, resources, and effort required for each task in the {project_type} project. You

## Create Crew, Agents and Tasks

In [9]:
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction


# Creating Agents
project_planning_agent = Agent(
  config=agents_config['project_planning_agent'],
  llm=llm
)

estimation_agent = Agent(
  config=agents_config['estimation_agent'],
  llm=llm
)

resource_allocation_agent = Agent(
  config=agents_config['resource_allocation_agent'],
  llm=llm,
)

# Creating Tasks
task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,  
  #output_format=ProjectPlan # This is the structured output we want
  
)

# Creating Crew
crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
  verbose=True,
)

## Crew's Inputs

In [10]:
from IPython.display import display, Markdown

project = 'Website'
industry = 'Technology'
project_objectives = 'Create a website for a small business'
team_members = """
- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Website

**Project Objectives:** Create a website for a small business

**Industry:** Technology

**Team Members:**

- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)

**Project Requirements:**

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust



## Kicking off the crew

In [11]:
# The given Python dictionary
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

# Agent: The Ultimate Project Planner
## Task: Carefully analyze the project_requirements for the Website project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build tru

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [12]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0018


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,12069,6858,5211,6


## Result

In [13]:
print(result)




```
{
    "tasks": [
        {
            "task_name": "Project Planning",
            "estimated_time_hours": 8,
            "required_resources": ["Project Manager"],
            "start_date": "2024-03-01",
            "end_date": "2024-03-02",
            "assigned_to": "John Doe"
        },
        {
            "task_name": "Design Website Layout",
            "estimated_time_hours": 16,
            "required_resources": ["Designer"],
            "start_date": "2024-03-03",
            "end_date": "2024-03-06",
            "assigned_to": "Bob Smith"
        },
        {
            "task_name": "Develop Website",
            "estimated_time_hours": 120,
            "required_resources": ["Software Engineer"],
            "start_date": "2024-03-07",
            "end_date": "2024-04-04",
            "assigned_to": "Jane Doe"
        },
        {
            "task_name": "Test Website",
            "estimated_time_hours": 40,
            "required_resources": ["QA Engineer"],
      

In [22]:
result.pydantic.dict()

AttributeError: 'NoneType' object has no attribute 'dict'

## Inspect further

In [ ]:
tasks = result.pydantic.dict()['tasks']
df_tasks = pd.DataFrame(tasks)

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

### Inspecting Milestones

In [ ]:
milestones = result.pydantic.dict()['milestones']
df_milestones = pd.DataFrame(milestones)

# Display the DataFrame as an HTML table
df_milestones.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)